# Yelp Restaurant Finder Agent

This notebook demonstrates how to create a restaurant finder agent using the Atomic Agents library. The agent will interact with users to gather their preferences and search for restaurants using the Yelp API.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/KennyVaneetvelde/atomic_agents/blob/main/examples/notebooks/yelp_agent.ipynb)


## Prerequisites

Before proceeding with this notebook, it is highly recommended to read up on the basics of the following libraries:

- **Pydantic**: A data validation and settings management library using Python type annotations. You can find more information and documentation at [Pydantic GitHub](https://github.com/pydantic/pydantic).
- **Instructor**: A Python library that simplifies working with structured outputs from large language models (LLMs). It provides a user-friendly API to manage validation, retries, and streaming responses. More details can be found at [Instructor GitHub](https://github.com/jxnl/instructor).

Understanding these libraries will help you make the most of this library.


## Step 1: Install Necessary Packages

First, we need to install the required packages. Run the following command to install `atomic-agents`, `openai`, and `instructor` libraries.

In [1]:
# Install necessary packages
%pip install atomic-agents openai instructor

Note: you may need to restart the kernel to use updated packages.


## Step 2: Import Libraries

We will import the necessary libraries for creating the restaurant finder agent.

In [2]:
import os
import logging
from typing import Union
from pydantic import BaseModel, Field
import instructor
import openai
from rich.console import Console

from atomic_agents.lib.components.agent_memory import AgentMemory
from atomic_agents.lib.components.system_prompt_generator import SystemPromptGenerator, SystemPromptInfo
from atomic_agents.agents.base_chat_agent import BaseChatAgent, BaseChatAgentOutputSchema, BaseChatAgentConfig
from atomic_agents.lib.tools.yelp_restaurant_finder_tool import YelpSearchTool, YelpSearchToolConfig, YelpSearchToolSchema

## Step 3: Define System Prompt Information

Define the system prompt information including background, steps, and output instructions.

In [3]:
# Define system prompt information including background, steps, and output instructions
system_prompt = SystemPromptInfo(
    background=[
        'This assistant is a restaurant finder AI designed to help users find the best restaurants based on their preferences by asking clarifying questions.',
    ],
    steps=[
        'Greet the user and introduce yourself as a restaurant finder assistant.',
        'Inspect the required Yelp schema and identify the necessary filters.',
        'Ask the user questions to gather information for each filter until all required information is clear.',
        'Use the chat responses to gather all necessary information from the user.',
        'Once all required information is gathered, use the YelpSearchTool schema to search Yelp for restaurants.',
    ],
    output_instructions=[
        'Always use the chosen schema as output',
        'Provide helpful and relevant information to assist the user.',
        'Be friendly and respectful in all interactions.',
        'Ensure that the chat responses are used to ask clarifying questions and gather information, and the Yelp schema is used to perform the actual search.'
    ]
)

## Step 4: Initialize Components

Initialize the system prompt generator, chat memory, and console for output.

In [4]:
# Initialize the system prompt generator with the defined system prompt and dynamic info providers
system_prompt_generator = SystemPromptGenerator(system_prompt)

# Initialize chat memory to store conversation history
memory = AgentMemory()
# Define initial memory with a greeting message from the assistant
initial_memory = [
    {'role': 'assistant', 'content': 'Hello, can I help you find a restaurant?'}
]
# Load the initial memory into the chat memory
memory.load(initial_memory)

console = Console()

## Step 5: Initialize the Client

Initialize the client using the `instructor` library. For all supported clients such as Anthropic & Gemini, refer to the `instructor` library documentation.

In [5]:
# Initialize the client
client = instructor.from_openai(openai.OpenAI())

## Step 6: Initialize the YelpSearchTool

Set up the YelpSearchTool with the necessary configuration.

In [6]:
# Initialize the YelpSearchTool
yelp_tool = YelpSearchTool(YelpSearchToolConfig(api_key=os.getenv('YELP_API_KEY'), max_results=10))

ValidationError: 1 validation error for YelpSearchToolConfig
api_key
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type

## Step 7: Define a Custom Response Schema

Create a custom response schema that can handle both chat responses and Yelp search tool responses.

In [ ]:
# Define a custom response schema that can handle both chat responses and Yelp search tool responses
class ResponseSchema(BaseModel):
    chosen_schema: Union[BaseChatAgentOutputSchema, YelpSearchToolSchema] = Field(..., description='The response from the chat agent.')

    class Config:
        title = 'OutputSchema'
        description = 'The response schema for the chat agent.'
        json_schema_extra = {
            'title': title,
            'description': description,
        }

: 

## Step 8: Create the Chat Agent

Create a chat agent with the specified model, system prompt generator, and memory.

In [ ]:
# Create a chat agent with the specified model, system prompt generator, and memory
agent = BaseChatAgent(
    config=BaseChatAgentConfig(
        client=client, 
        system_prompt_generator=system_prompt_generator,
        model='gpt-3.5-turbo',
        memory=memory,
        output_schema=OutputSchema
    )
)

console.print("BaseChatAgent with YelpSearchTool is ready.")
console.print(f'Agent: {initial_memory[0]["content"]}')

: 

## Step 9: Run the Agent

Start the agent to interact with the user. The agent will ask for user input, process it, and provide responses based on the gathered information.
Try telling it you would like some pizza, it should ask for your location and other information until it feels it can properly call the Yelp API with the information it has gathered.

In [ ]:
while True:
    user_input = input('You: ')
    if user_input.lower() in ['exit', 'quit']:
        print('Exiting chat...')
        break

    response = agent.run(agent.input_schema(chat_message=user_input).model_dump())
    
    # Check the type of the response schema
    if isinstance(response.chosen_schema, YelpSearchToolSchema):
        output = yelp_tool.run(response.chosen_schema)
        agent.memory.add_message('system', f'The following information was found: {output.results}\n\n Please summarize the results for the user.')
        output = agent.run().chosen_schema.chat_message
    else:
        output = response.chosen_schema.chat_message
        
    console.print(f'Agent: {output}')

: 

## Conclusion

In this notebook, we demonstrated how to create a restaurant finder agent using the Atomic Agents library. The agent interacts with users to gather their preferences and searches for restaurants using the Yelp API. You can further customize the agent and enhance its capabilities based on your requirements.